<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import hashlib
import requests
import sqlite3
import pandas as pd
from google.colab import userdata

PUBLIC_KEY = userdata.get('PUBLIC_KEY')
PRIVATE_KEY = userdata.get('PRIVATE_KEY')

# Criando o hash MD5
hashtre = hashlib.md5(str.encode(str(1) + str(PRIVATE_KEY) + str(PUBLIC_KEY)))

# Base URL da API da Marvel
base = "https://gateway.marvel.com"
url_montada_characters = f"{base}/v1/public/characters?ts=1&apikey={PUBLIC_KEY}&hash={hashtre.hexdigest()}"

# Fazendo a requisição GET para a API e obtendo os dados
response = requests.get(url_montada_characters)

# Convertendo a resposta para JSON
dados = response.json()

# Acessando os resultados dos personagens
descricao = dados["data"]["results"]

# Conectando ao banco de dados SQLite (cria o banco se não existir)
conn = sqlite3.connect('marvel_characters.db')
cursor = conn.cursor()

# Criando a tabela para armazenar personagens
cursor.execute('''CREATE TABLE IF NOT EXISTS personagens (
                    id INTEGER PRIMARY KEY,
                    name TEXT,
                    description TEXT)''')

# Criando a tabela para armazenar quadrinhos
cursor.execute('''CREATE TABLE IF NOT EXISTS comics (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    character_id INTEGER,
                    comic_name TEXT,
                    FOREIGN KEY (character_id) REFERENCES personagens (id))''')

# Inserindo os dados dos personagens e quadrinhos no banco de dados
for char in descricao:
    char_id = char["id"]
    char_name = char["name"]
    char_description = char["description"]

    # Inserir personagem na tabela "personagens"
    cursor.execute('''INSERT OR IGNORE INTO personagens (id, name, description)
                      VALUES (?, ?, ?)''', (char_id, char_name, char_description))

    # Inserir quadrinhos associados ao personagem
    for comic in char["comics"]["items"]:
        comic_name = comic["name"]
        cursor.execute('''INSERT INTO comics (character_id, comic_name)
                          VALUES (?, ?)''', (char_id, comic_name))

# Commit e fechar a conexão
conn.commit()

# Lendo as tabelas com pandas
personagens_df = pd.read_sql_query("SELECT * FROM personagens", conn)
comics_df = pd.read_sql_query("SELECT * FROM comics", conn)

# Fechar a conexão
conn.close()

# Exibindo as tabelas
print("Tabela de Personagens:")
print(personagens_df)  # Exibe as primeiras linhas da tabela de personagens

print("\nTabela de Quadrinhos:")
print(comics_df)  # Exibe as primeiras linhas da tabela de quadrinhos


Tabela de Personagens:
         id                        name  \
0   1009144                      A.I.M.   
1   1009146  Abomination (Emil Blonsky)   
2   1009148               Absorbing Man   
3   1009149                       Abyss   
4   1009150                  Agent Zero   
5   1010354                Adam Warlock   
6   1010699                 Aaron Stack   
7   1010846        Aegis (Trey Rollins)   
8   1010903   Abyss (Age of Apocalypse)   
9   1011031              Agent X (Nijo)   
10  1011136    Air-Walker (Gabriel Lan)   
11  1011175                      Aginar   
12  1011198             Agents of Atlas   
13  1011266                Adam Destine   
14  1011297                 Agent Brand   
15  1011334                     3-D Man   
16  1012717             Agatha Harkness   
17  1016823      Abomination (Ultimate)   
18  1017100                A-Bomb (HAS)   
19  1017851                 Aero (Aero)   

                                          description  
0   AIM is a terr